# Hi‑Tech VC Valuation — Notebook

This notebook computes VC‑method valuations for Galaxy and Acorn, including:
- Initial valuation implied by term sheets (with 2nd round dilution modeled)
- Final proposed valuation (via adjustable term versions)
- Sensitivity analysis (e.g., vs. target IRR)
- Waterfall analysis for M&A vs IPO (mandatory conversion at IPO)

All calculations reuse the same Python model used by the API and React UI.



In [ ]:
# Setup and imports
import os, sys, json
from pathlib import Path

# Ensure local backend package is importable
sys.path.append(str(Path('.').resolve()))

from backend.models.terms import load_terms
from backend.models.valuation import (
    project_exit, exit_value_from_pe, solve_premoney_for_target_irr,
    simulate_rounds, investor_proceeds_and_irr, waterfall
)

import ipywidgets as w
from IPython.display import display, Markdown

BASE_YEAR = 2030
BASE_REVENUE = 52_000_000.0
BASE_NET_INCOME = 3_450_000.0
DEFAULT_MARGIN = BASE_NET_INCOME / BASE_REVENUE



In [ ]:
def vc_compute(
    investor: str,
    version: str | None,
    cagr: float,
    margin: float,
    pe_multiple: float | None,
    exit_horizon_years: int | None,
    target_irr: float | None,
    follow_on_amount: float | None,
    follow_on_year: int | None,
    series_b_multiple: float | None,
    pro_rata: float | None,
    scenario: str = 'M&A'
):
    terms = load_terms(investor, version=version)
    # Apply overrides
    if follow_on_amount is not None:
        terms.follow_on.investment_amount = float(follow_on_amount)
    if follow_on_year is not None:
        terms.follow_on.year_offset = int(follow_on_year)
    if series_b_multiple is not None:
        terms.follow_on.series_b_price_multiple = float(series_b_multiple)
    if pro_rata is not None:
        terms.follow_on.pro_rata_participation = float(pro_rata)

    years = int(exit_horizon_years or terms.default_exit_horizon_years)
    pe = float(pe_multiple if pe_multiple is not None else terms.default_pe_multiple)
    irr_target = float(target_irr if target_irr is not None else terms.target_irr)

    exit_rev, exit_ni = project_exit(cagr, margin, terms.base_revenue, terms.base_year, terms.base_year + years)
    exit_equity = exit_value_from_pe(exit_ni, pe)

    pre_money, pps, inv_exit = solve_premoney_for_target_irr(terms, exit_equity, years, irr_target, scenario=scenario)

    rounds = simulate_rounds(terms, pps)
    post_money_a = pre_money + terms.total_a_investment
    owner_post_a = rounds.series_a_shares_investor / rounds.post_money_shares_after_a

    return {
        'investor': investor,
        'version': version or 'active',
        'pre_money': pre_money,
        'price_per_share': pps,
        'post_money_after_a': post_money_a,
        'investor_ownership_post_a': owner_post_a,
        'investor_exit_cash': inv_exit,
        'exit_equity_value': exit_equity,
        'years': years,
        'inputs_used': {
            'cagr': cagr, 'margin': margin, 'pe_multiple': pe, 'target_irr': irr_target
        }
    }



In [ ]:
# Interactive inputs
inv = w.ToggleButtons(options=['galaxy', 'acorn'], description='Investor:', value='galaxy')
ver = w.Text(value='', description='Version (blank=active)')
cagr = w.BoundedFloatText(value=0.25, min=0.0, max=2.0, step=0.01, description='CAGR')
margin = w.BoundedFloatText(value=0.15, min=0.0, max=1.0, step=0.005, description='Net margin')
pe = w.BoundedFloatText(value=0.0, min=0.0, max=500.0, step=1.0, description='P/E (0=default)')
exit_years = w.BoundedIntText(value=0, min=0, max=15, description='Exit years (0=default)')
irr = w.BoundedFloatText(value=0.0, min=0.0, max=2.0, step=0.01, description='Target IRR (0=default)')
fol_amt = w.BoundedFloatText(value=0.0, min=0.0, max=1e9, step=100000, description='Follow-on $ (0=def)')
fol_year = w.BoundedIntText(value=0, min=0, max=15, description='Follow-on year (0=def)')
sb_mult = w.BoundedFloatText(value=2.0, min=0.1, max=10.0, step=0.1, description='Series B multiple')
pro_rata = w.BoundedFloatText(value=1.0, min=0.0, max=1.0, step=0.05, description='Pro-rata (0-1)')
scenario = w.ToggleButtons(options=['M&A', 'IPO'], description='Scenario:', value='M&A')

ui = w.VBox([inv, ver, cagr, margin, pe, exit_years, irr, fol_amt, fol_year, sb_mult, pro_rata, scenario])

out = w.Output()

def render(_=None):
    out.clear_output()
    v = vc_compute(
        inv.value,
        ver.value or None,
        cagr.value,
        margin.value,
        (None if pe.value == 0 else pe.value),
        (None if exit_years.value == 0 else exit_years.value),
        (None if irr.value == 0 else irr.value),
        (None if fol_amt.value == 0 else fol_amt.value),
        (None if fol_year.value == 0 else fol_year.value),
        (None if sb_mult.value == 0 else sb_mult.value),
        (None if pro_rata.value < 0 else pro_rata.value),
        scenario.value
    )
    with out:
        display(Markdown(f"""
### VC Method — {v['investor'].upper()} (version: {v['version']})
- Exit Equity Value: ${v['exit_equity_value']:,.0f}
- Pre‑Money Valuation: ${v['pre_money']:,.0f}
- Price per Share: ${v['price_per_share']:,.4f}
- Post‑Money (after A): ${v['post_money_after_a']:,.0f}
- Investor Ownership (post A): {v['investor_ownership_post_a']*100:.2f}%
- Investor Exit Cash: ${v['investor_exit_cash']:,.0f}
"""))

for wdg in [inv, ver, cagr, margin, pe, exit_years, irr, fol_amt, fol_year, sb_mult, pro_rata, scenario]:
    wdg.observe(render, names='value')

render()
display(w.HBox([ui, out]))



In [ ]:
# Sensitivity vs Target IRR (fixed other inputs)
import pandas as pd

def sensitivity_irr(investor: str, version: str | None, cagr: float, margin: float, pe_multiple: float | None, exit_years: int | None, irr_points=(0.3,0.4,0.5,0.6,0.7)):
    rows = []
    for x in irr_points:
        res = vc_compute(investor, version, cagr, margin, pe_multiple, exit_years, x, None, None, None, None)
        rows.append({ 'target_irr': x, 'pre_money': res['pre_money'], 'price_per_share': res['price_per_share'] })
    return pd.DataFrame(rows)

_df = sensitivity_irr(inv.value, ver.value or None, cagr.value, margin.value, (None if pe.value==0 else pe.value), (None if exit_years.value==0 else exit_years.value))
_df



In [ ]:
# Waterfall analysis
exit_value = w.BoundedFloatText(value=200_000_000.0, min=10_000_000.0, max=2_000_000_000.0, step=1_000_000.0, description='Exit Equity $')
wf_out = w.Output()

def render_wf(_=None):
    wf_out.clear_output()
    v = vc_compute(inv.value, ver.value or None, cagr.value, margin.value, (None if pe.value==0 else pe.value), (None if exit_years.value==0 else exit_years.value), (None if irr.value == 0 else irr.value), fol_amt.value or None, fol_year.value or None, sb_mult.value or None, pro_rata.value or None, scenario.value)
    from backend.models.valuation import simulate_rounds, waterfall
    terms = load_terms(inv.value, version=ver.value or None)
    rounds = simulate_rounds(terms, v['price_per_share'])
    payouts = waterfall(exit_value.value, terms, rounds, v['years'], scenario=scenario.value)
    with wf_out:
        display(Markdown(f"""
### Waterfall — {inv.value.upper()} ({scenario.value})
- Exit Equity Value: ${exit_value.value:,.0f}
- Price per Share (A): ${v['price_per_share']:,.4f}

| Class | Payout |
|---|---:|
| Series A Investor | ${payouts['series_a_investor']:,.0f} |
| Series A Others | ${payouts['series_a_others']:,.0f} |
| Common | ${payouts['common']:,.0f} |
"""))

for wdg in [inv, ver, cagr, margin, pe, exit_years, irr, fol_amt, fol_year, sb_mult, pro_rata, scenario, exit_value]:
    wdg.observe(render_wf, names='value')

render_wf()
display(w.VBox([exit_value, wf_out]))



## Notes and assumptions
- Series B is modeled to capture dilution and optional pro‑rata participation for the Series A investor; no separate liquidation preference is modeled for Series B.
- Dividends are modeled as simple cumulative at 10% per year for Series A; paid at liquidity (M&A). In IPO scenarios, preferred shares mandatorily convert to common and preferences/dividends are not paid.
- You can switch investor (Galaxy/Acorn), set version, and adjust CAGR/margin/IRR/P‑E/exit years and follow‑on levers. Sensitivity and waterfall sections update accordingly.

